In [1]:
from jwst_SNR import jwst_SNRclass
import numpy as np
from nirspec_sn import *
%matplotlib notebook

In [2]:
from background4jwst import background4jwstclass
background4jwst=background4jwstclass()
    
# set the position for which the background will be calculated. set_position_by_target uses 
# pre-defined positions, set_position can be used to set any position
print(background4jwst.defaulttargets)
background4jwst.set_position_by_target('ElGordo')
# alternatively, background4jwst.set_position('01 02 55.2','-49 14 29.3','ElGordo')
# Get the background from pandeia for this position
# optional parameters: lam, thresh
background4jwst.calc_background()

{'ElGordo': ('01 02 55.2', '-49 14 29.3'), 'EmptyERS': ('03 32 42.397', '-27 42 7.93'), 'NEP-TDF': ('17:22:47.896', '+65:49:21.54'), 'NEP-DF': ('17:40:08.00', '+69:00:08.00'), 'CDF-S': ('03:32:28.0', '−27:48:30')}
Calculating background for position ElGordo (15.730000,-49.241472)


In [3]:
jwst_SNR=jwst_SNRclass(instrument='nirspec')
#jwst_SNR=jwst_SNRclass(instrument='nircam')

jwst_SNR.set_background4jwst(50,target='EmptyERS')

Initializing pandeia with nirspec, None
Initializing nirspec fixed_slit
Calculating background for position EmptyERS (53.176654,-27.702203)
### Calculating percentile: 50
day: 187


0

In [4]:
from astropy.io import ascii

file = './data/siebert_phase_binned_2day_N=99_Nspec=153_phase=p0.95_dm15=1.15_z=0.014.txt'
data = ascii.read(file)  
df = data.to_pandas()
snia = S.ArraySpectrum(df['Wavelength'].values,df['Flux'].values,waveunits='angstrom',
               fluxunits='flam')
snia = snia.redshift(4)

Warning, 26 of 4762 bins contained negative fluxes; they have been set to zero.


In [5]:
# this part fials
jwst_SNR.verbose=2
jwst_SNR.Imaging_texp_table(['F200W'],np.arange(24.0,30.0,1),5,spec=snia,wave=2,width=.5)
jwst_SNR.texp.write('NIRSpec_SNR4.txt',formatters=jwst_SNR.formatters4texptable)

#############################
#### Filter F200W, mag 24.00 for S/N=5 
#############################
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 16, 'NINT': 1, 'tint': 233.4, 'NEXP': 4, 'texp': 992.0}


total texp  992.0445199999999
filter:F200W mag:24.00, target exptime:1000.0  ==> SNR=13.20 exptime=992.0
SNR= 13.20 for starting texp= 992.0
texp guess: 261.1
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 4, 'NINT': 1, 'tint': 58.4, 'NEXP': 4, 'texp': 291.8}


total texp  291.77779999999996
filter:F200W mag:24.00, target exptime:291.8  ==> SNR=2.68 exptime=291.8
SNR=  2.68 for next texp= 291.8
SNR=  2.68<5.00 for texp= 291.8, checking the next larger exptime...
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 8, 'NINT': 1, 'tint': 116.7, 'NEXP': 4, 'texp': 525.2}


total texp  525.20004
filter:F200W mag:24.00, target exptime:525.2  ==> SNR=6.58 exptime=525.2
SNR=  6.58>=5.00 for texp= 525.2!! SUCCESS!
#############################
#### Filter F200W, mag 25.00 for S/N=5 
#############################
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 16, 'NINT': 1, 'tint': 233.4, 'NEXP': 4, 'texp': 992.0}


total texp  992.0445199999999
filter:F200W mag:25.00, target exptime:1000.0  ==> SNR=5.89 exptime=992.0
SNR=  5.89 for starting texp= 992.0
texp guess: 791.7
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 16, 'NINT': 1, 'tint': 233.4, 'NEXP': 4, 'texp': 992.0}


...

total texp  992.0445199999999
filter:F200W mag:28.00, target exptime:1000.0  ==> SNR=0.41 exptime=992.0
SNR=  0.41 for starting texp= 992.0
texp guess: 31324.2
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 84, 'NINT': 2, 'tint': 1225.5, 'NEXP': 4, 'texp': 9862.2}


total texp  9920.4452
filter:F200W mag:28.00, target exptime:9862.2  ==> SNR=1.94 exptime=9920.4
SNR=  1.94 for next texp=9920.4
SNR=  1.94<5.00 for texp=9920.4, checking the next larger exptime...
#############################
#### Filter F200W, mag 29.00 for S/N=5 
#############################
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 16, 'NINT': 1, 'tint': 233.4, 'NEXP': 4, 'texp': 992.0}


total texp  992.0445199999999
filter:F200W mag:29.00, target exptime:1000.0  ==> SNR=0.16 exptime=992.0
SNR=  0.16 for starting texp= 992.0
texp guess: 110538.2
reference spec
Ehu  {'readout_pattern': 'nrsirs2rapid', 'NGROUP': 84, 'NINT': 2, 'tint': 1225.5, 'NEXP': 4, 'texp': 9862.2}


total texp  9920.4452
filter:F200W mag:29.00, target exptime:9862.2  ==> SNR=0.78 exptime=9920.4
SNR=  0.78 for next texp=9920.4
SNR=  0.78<5.00 for texp=9920.4, checking the next larger exptime...


0

In [ ]:
jwst_SNR.ETCresults

In [ ]:
jwst_SNR.ETCresults

In [ ]:
plt.figure()
plt.plot(snia.wave,snia.flux)

In [ ]:
jwst_SNR.verbose=2
jwst_SNR.Imaging_SNR_table(['F200W','F444W'],np.arange(25.0,26.0,0.5),1000,spec=snia,wave=2,width=.5)

In [ ]:
import math

In [ ]:
math.pow(5/2,2)

In [ ]:
res = jwst_SNR.ETCresults

In [ ]:
snr = res['1d']['sn']

In [ ]:
plt.figure()
plt.plot(snr[0],snr[1])

In [ ]:
plt.axvspan()